# Testing how to simulate model quality inferences

## Section 1 - Setup

><div class="alert alert-block alert-info"><b>NOTE: </b>Recommend using an <em>ml.m5.large</em> instance type and, <em>Python 3 (Data Science)</em> kernel to train the <b>CTGAN</b> model.</div

In [1]:
# Install CTGAN
#!pip install CTGAN

In [2]:
from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
import io
import requests
import tempfile

import warnings

import pandas as pd
import numpy as np


from time import sleep
from threading import Thread

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()

In [3]:
# Get Execution role
role = get_execution_role()
print("RoleArn:", role)

region = session.boto_region_name
print("Region:", region)

RoleArn: arn:aws:iam::500842391574:role/SageMaker
Region: us-east-2


In [4]:
# Setup S3 bucket parmaters for the production logs bucket
bucket = 'proddeploymentstage-prodappl-logss3bucket004b0f70-1y2budxvotxwc'
print("Inference Testing Bucket:", bucket)

# S3 prefixes
data_capture_prefix = 'endpoint-data-capture'
s3_capture_upload_path = f's3://{bucket}/{data_capture_prefix}'
ground_truth_upload_path = f's3://{bucket}/ground-truth-data/{datetime.now():%Y-%m-%d-%H-%M-%S}'

# Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")

Inference Testing Bucket: proddeploymentstage-prodappl-logss3bucket004b0f70-1y2budxvotxwc
Image URI: 777275614652.dkr.ecr.us-east-2.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://proddeploymentstage-prodappl-logss3bucket004b0f70-1y2budxvotxwc/endpoint-data-capture
Ground truth path: s3://proddeploymentstage-prodappl-logss3bucket004b0f70-1y2budxvotxwc/ground-truth-data/2021-05-02-18-26-22


---

## Section 2 - Recreate Baseline Data

We will re-create the Model Quality baseline job (even though it was already created by the CDK Pipeline) to see the output of the SageMaker SDK when calling `create_monitoring_schedule()`, as well as, to leverage the resultant constraints when creating the monitoring schedule.

In [5]:
# Set up the locations for capturing the baseline results
# This should already be in place from the CDK Pipeline,
# with the `baseline.csv` already there
baseline_prefix = 'baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_dataset_uri = f's3://{bucket}/{baseline_data_prefix}'
baseline_results_uri = f's3://{bucket}/{baseline_results_prefix}'
print(f'Baseline data uri: {baseline_dataset_uri}')
print(f'Baseline results uri: {baseline_results_uri}')

Baseline data uri: s3://proddeploymentstage-prodappl-logss3bucket004b0f70-1y2budxvotxwc/baselining/data
Baseline results uri: s3://proddeploymentstage-prodappl-logss3bucket004b0f70-1y2budxvotxwc/baselining/results


In [6]:
# Generate a new baseline job
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat

# Create the model quality monitoring object
model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session
)

In [7]:
# Name of the model quality baseline job
baseline_job_name = f'abalone-baseline-{datetime.utcnow():%Y-%m-%d-%H%M}'

In [ ]:
# Execute the baseline suggestion job. 
# Specify problem type, in this case Regression, and provide other required attributes.
job = model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri = baseline_results_uri,
    problem_type='Regression',
    inference_attribute= "prediction",
    ground_truth_attribute= "label"
)
job.wait(logs=False)

### Explore the Generated Metrics

In [ ]:
baseline_job = model_quality_monitor.latest_baselining_job
binary_metrics = baseline_job.baseline_statistics().body_dict["regression_metrics"]
pd.json_normalize(binary_metrics).T

### Explore the Generated Constraints

In [ ]:
pd.DataFrame(baseline_job.suggested_constraints().body_dict["regression_constraints"]).T

---

## Section 3 - Create Inferene Data to Test the Model Quality Monitor

Model Quality Monitoring needs two additional inputs - predictions made by the deployed model endpoint and the ground truth data to be provided by the model consuming application. Since you already enabled data capture on the endpoint, prediction data is captured in S3. The ground truth data depends on the what the model is predicting and what the business use case is.

### Gerating Synthetic Abalone data

In order to generate prediction data we will need to create fake "new" data. To accomplish this, we will use the CTGAN package and train it on the "raw" abaloen dataset. We will create $300$ samples of fake data.

In [8]:
import warnings

warnings.filterwarnings('ignore')
s3 = boto3.client('s3')

# 'raw' data column names
names = [
    'sex',
    'length',
    'diameter',
    'height',
    'whole_weight',
    'shucked_weight',
    'viscera_weight',
    'shell_weight',
    'rings'
]

# Location of the 'raw' data
bucket = 'data-us-east-2-500842391574'
key = 'input/raw/abalone.csv'
obj = s3.get_object(Bucket=bucket, Key=key)
raw_data = pd.read_csv(io.BytesIO(obj['Body'].read()), encoding='utf8', names=names)
raw_data.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [9]:
from ctgan import CTGANSynthesizer

# Fit the CTGAN model, declaring the 'sex'column as discrete variables
ctgan = CTGANSynthesizer()
ctgan.fit(raw_data, ['sex'])

In [10]:
# Generate 300 samples from the CTGAN model
samples = ctgan.sample(300)

In [11]:
# Compare the raw data
raw_data.describe()

,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [12]:
# Compare the sample data
samples.describe()

,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,0.541224,0.403590,0.154495,0.725746,0.382409,0.203390,0.221105,9.493333
std,0.140408,0.127399,0.045346,0.492354,0.225591,0.126288,0.142308,3.309552
min,0.101786,0.035434,-0.005030,-0.113143,-0.030926,-0.019424,-0.026072,3.000000
25%,0.457475,0.331583,0.129630,0.397777,0.223960,0.107950,0.118371,8.000000
50%,0.549092,0.424468,0.158490,0.725064,0.348765,0.198957,0.205575,9.000000
75%,0.642550,0.497184,0.186664,1.015252,0.508372,0.296405,0.318155,11.250000
max,0.786797,0.633748,0.243380,2.400230,1.254820,0.572425,0.840583,22.000000


In [13]:
# Save the samples as fake abalone data
samples.to_csv('fake-abalone.csv', index=False)

### Pre-Process the Synthetic Abalone Data

Since the synthetic data is based on the "raw" Abalone dataset and, in order to execute inferences against the production endpoint, we will need to transform the data into to the format our model has been trained upon. To do this, we will follow the same proceedure for preprocessing the origional Abalone data.

```python
import os
import requests
import tempfile
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

feature_columns_dtype = {
    'sex': str,
    'length': np.float64,
    'diameter': np.float64,
    'height': np.float64,
    'whole_weight': np.float64,
    'shucked_weight': np.float64,
    'viscera_weight': np.float64,
    'shell_weight': np.float64
}
label_column_dtype = {'rings': np.float64}


def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)
    return z


# Preprocess the data
df = pd.read_csv(
    'fake-abalone.csv',
    dtype=merge_two_dicts(feature_columns_dtype, label_column_dtype)
)
df
```

```python
# Since we get a headerless CSV file we specify the column names here.
feature_columns_names = [
    'sex',
    'length',
    'diameter',
    'height',
    'whole_weight',
    'shucked_weight',
    'viscera_weight',
    'shell_weight',
]
label_column = 'rings'


# Set up the processing pipeline for Numerical feature scaling
numeric_features = list(feature_columns_names)
numeric_features.remove('sex')
numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

# Set up the processing pipeline for categorical encoding
categorical_features = ['sex']
categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

# Create the transformer
preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Preprocess the data
y = df.pop('rings')
X_pre = preprocess.fit_transform(df)
y_pre = y.to_numpy().reshape(len(y), 1)

header = [
    'length',
    'diameter',
    'height',
    'whole_weight',
    'shucked_weight',
    'viscera_weight',
    'shell_weight',
    'sex_F',
    'sex_I',
    'sex_M'
]

# Create the inference dataset
# pd.DataFrame(X_pre).to_csv('inference-data.csv', header=False, index=False)
pd.DataFrame(X_pre).to_csv('inference-data.csv', header=header, index=False)

# Create the ground trutch dataset
pd.DataFrame(y_pre).to_csv('ground-truth.csv', header=['label'], index=False)
```

We should now have the following syntehtic data files:
- `fake-abalone.csv`: Synthic "raw" abalone data.
- `inference-data.csv`: Preprocessed data to generate predictions from the Production Endpoint.
- `ground-truth.csv`: The ground truth labels from the synthetic data wich to compare the quality of the model's predictions.

---

## Section 4 - Setup continuous model monitoring to identify model quality drift 

### Load the Data

### Generate Inferences uding the `FormHandler` API

In [ ]:
# def invoke_endpoint(ep_name, file_name):    
#     with open(file_name, 'r') as f:
#         i = 0
#         for row in f:
#             payload = row.rstrip('\n')
#             response = session.sagemaker_runtime_client.invoke_endpoint(
#                 EndpointName=endpoint_name,
#                 ContentType='text/csv', 
#                 Body=payload,
#                 InferenceId=str(i), # unique ID per row
#             )["Body"].read()
#             i += 1
#             sleep(1)
            
# def invoke_endpoint_forever():
#     while True:
#         invoke_endpoint(endpoint_name, 'inference-data.csv')
        
# thread = Thread(target = invoke_endpoint_forever)
# thread.start()

View captured data stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [123]:
print("Waiting for captures to show up", end="")
for _ in range(300): #5 Minutes
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    sleep(1)
print()
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))

Waiting for captures to show up

NameError: name 'endpoint_name' is not defined

View the contents of a single capture file. Here you should see all the data captured in an Amazon SageMaker specific JSON-line formatted file. Take a quick peek at the first few lines in the captured file.

In [ ]:
print("\n".join(capture_file[-3:-1]))

View the contents of a single line is present below in a formatted JSON file so that you can observe a little better.

In [ ]:
print(json.dumps(capture_record, indent=2))

---

# Prototyping: Load the Fake Data

## Simulate the __original__ form request

In [111]:
df = pd.read_csv('inference-data.csv')
df

,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,sex_F,sex_I,sex_M
0,1.030321,0.625345,-0.510023,-0.450467,0.967696,0.999419,0.044743,1.0,0.0,0.0
1,1.002049,0.786466,1.001100,0.618082,-0.152741,0.580372,0.328322,1.0,0.0,0.0
2,0.016244,0.820966,-0.197747,-0.794957,-0.589634,-0.228903,-0.567400,0.0,1.0,0.0
3,0.349017,-0.596164,0.063278,-0.144823,-0.197446,0.964422,-0.722974,1.0,0.0,0.0
4,-0.657094,-0.250455,-0.939538,-0.121472,0.071549,-1.062272,1.034768,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
295,0.823343,0.713829,0.693015,0.416776,-0.182791,0.692146,1.319292,1.0,0.0,0.0
296,0.878026,0.215635,0.395211,0.463293,1.352296,0.077992,0.433733,0.0,0.0,1.0
297,-0.294669,-0.448046,-0.248960,0.338348,0.341708,0.653577,1.098421,0.0,0.0,1.0
298,-0.626096,-0.702165,-0.503231,-0.403328,-1.242498,-1.525372,-0.422840,0.0,1.0,0.0


In [113]:
#body = json.dumps(json.loads(df['json'][0]))
body = json.loads(df.iloc[0].to_json())
body

{'length': 1.0303210547,
 'diameter': 0.6253454581,
 'height': -0.5100226899,
 'whole_weight': -0.4504671329,
 'shucked_weight': 0.9676957699,
 'viscera_weight': 0.9994186984,
 'shell_weight': 0.0447432706,
 'sex_F': 1.0,
 'sex_I': 0.0,
 'sex_M': 0.0}

In [114]:
# df['json'] = df.apply(lambda x: x.to_json(), axis=1)
# body = json.loads(df['json'][0])
# body

In [115]:
api = 'https://3fdb5qj5qf.execute-api.us-east-2.amazonaws.com/'+'api/predict'
api

'https://3fdb5qj5qf.execute-api.us-east-2.amazonaws.com/api/predict'

In [116]:
headers = {"content-type":"application/json; charset=UTF-8", "inference-id":"0"}
response = requests.post(api, headers=headers, data = json.dumps(body))

In [117]:
response

<Response [200]>

In [121]:
def invoke_api(api, file_name):
    df = pd.read_csv(file_name)
    i = 0
    for row in range(len(df)):
        headers = {"content-type":"application/json; charset=UTF-8", "inference-id": str(i)}
        body = json.loads(df.iloc[row].to_json())
        response = requests.post(api, headers=headers, data = json.dumps(body))
        i += 1
        sleep(1)

In [122]:
invoke_api('https://3fdb5qj5qf.execute-api.us-east-2.amazonaws.com/'+'api/predict', 'inference-data.csv')

In [125]:
print("Waiting for captures to show up", end="")
for _ in range(10):
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/abalone-prod-endpoint"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    sleep(1)
print()
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))

Waiting for captures to show up..........
Found Capture Files:
s3://proddeploymentstage-prodappl-logss3bucket004b0f70-1y2budxvotxwc/endpoint-data-capture/abalone-prod-endpoint/AllTraffic/2021/05/02/19/31-15-074-8ac7f32d-3d86-4be7-974f-f7c8d19f5801.jsonl
 s3://proddeploymentstage-prodappl-logss3bucket004b0f70-1y2budxvotxwc/endpoint-data-capture/abalone-prod-endpoint/AllTraffic/2021/05/02/19/32-14-822-3211d613-80b8-4e49-8bba-73f8a3fb175f.jsonl
 s3://proddeploymentstage-prodappl-logss3bucket004b0f70-1y2budxvotxwc/endpoint-data-capture/abalone-prod-endpoint/AllTraffic/2021/05/02/19/32-15-945-c2853a92-9f78-45ff-af6f-ca75f522c77a.jsonl


In [126]:
print("\n".join(capture_file[-3:-1]))

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"-0.2946686172,-0.4480458169,-0.2489597399,0.3383480237,0.3417079097,0.6535765649,1.0984211402,0.0,0.0,1.0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"11.083495","encoding":"CSV"}},"eventMetadata":{"eventId":"d16d55ff-a9e7-43c0-b5a0-a33c250f004f","inferenceTime":"2021-05-02T19:32:36Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"-0.6260961569,-0.7021653362,-0.5032310101,-0.4033278206,-1.2424982885,-1.5253719385,-0.4228395077,0.0,1.0,0.0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"21.360489","encoding":"CSV"}},"eventMetadata":{"eventId":"85770182-7b50-48a7-b225-93589603227b","inferenceTime":"2021-05-02T19:32:37Z"},"eventVersion":"0"}


In [127]:
print(json.dumps(capture_record, indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "0.9513159831,0.6444342339,0.1541224377,0.5611244216,-0.360546775,1.0808794014,-0.706852323,0.0,0.0,1.0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "16.620396",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "b40ad35e-8daa-4a03-ac17-068a235b2041",
    "inferenceTime": "2021-05-02T19:32:15Z"
  },
  "eventVersion": "0"
}


In [129]:
pd.__version__

'1.1.4'

In [131]:
np.__version__

'1.20.2'